# Crypto-PAn(Cryptography-based  Prefix-preserving Anonymization) アルゴリズムの理解


prefix preserving な　IPアドレス匿名化手法

参考：
- http://www.cc.gatech.edu/computing/Networking/projects/cryptopan/
- Jinliang Fan, Jun Xu, Mostafa H. Ammar, Sue B. Moon, Prefix-preserving IP address anonymization: measurement-based security evaluation and a new cryptography-based scheme, Computer Networks, Volume 46, Issue 2, 7 October 2004, Pages 253-272, ISSN 1389-1286, http://dx.doi.org/10.1016/j.comnet.2004.03.033.
- http://www.cc.gatech.edu/computing/Networking/projects/cryptopan/Crypto-PAn.1.0.tar.gz
- https://github.com/keiichishima/yacryptopan



In [5]:
import Crypto.Cipher.AES
import socket
import struct
from array import array
import ipaddress
import numpy as np

#Orig IP: 192.168.0.1
#Ano IP: 2.149.253.242
  

class CryptoPAn:
    def __init__(self,key):
        """ コンストラクタ
        :param key: 暗号化用バイト配列（３２バイト）（上位１６バイトは暗号鍵，下位１６バイトはパディング用）
        :type key: array 
        """
        self._init(key)

    def _init(self,key):
        self._cipher = Crypto.Cipher.AES.new(key[:16],Crypto.Cipher.AES.MODE_ECB)
        padding = array('B')
        padding.fromstring(self._cipher.encrypt(key[16:]))
        self._padding_int = self._to_int(padding)
        
    def changeKey(slef,key):
        """暗号化鍵を変更する
        """
        self._init(key)
    
    def MSB(self,byte_array):
        """最上位ビットを取得する
        """
        n = byte_array[0] >> 7
        return n
    
    def LSB(self,byte_array):
        """最下位ビットを取得する
        """
        n = self._to_int(byte_array)
        return n & 1

    def anonymize(self,orig_addr,priv):
        """ IPアドレスを匿名化する
        :param orig_addr: IPアドレスの文字列
        :type orig_addr: string
        :type priv: int
        :return anonymizedIP: int
        :rtype anonymizedIP: int
        """
        # orig_addr を unicode にする
        if not isinstance(orig_addr,unicode):
            orig_addr = unicode(orig_addr)
            
        ip=ipaddress.ip_address(orig_addr)
        addr = int(ip)
        if ip.version == 4:
            N = 32
            ext_addr = addr << 96
        elif ip.version == 6:
            N = 128
            ext_addr = addr 
         
        flip_array = []
        
        if priv > N:
            priv = N
        
        for pos in range(N):
            prefix =  ext_addr >> (128-pos) << (128-pos) 
            padded_addr = prefix | (self._padding_int & (2**128-1 >> pos))
            self._input = self.__to_byte_array(padded_addr,16)
            output = array('B')
            output.fromstring(self._cipher.encrypt(self._input))
            # 論文だとLSBだけど実装はMSB
            flip_array.append(self.MSB(output))
        result = reduce(lambda x, y: (x << 1) | y, flip_array)
        
        anonymizedIP = addr ^ (result & (2**N-1<<(N-priv)))
        if (ip.version == 4):
            anonymizedIP = ipaddress.IPv4Address(anonymizedIP)
        elif (ip.version  ==6):
            anonymizedIP = ipaddress.IPv6Address(anonymizedIP) 
        return anonymizedIP
    
    def _to_int(self,byte_array):
        """ byte配列をint型に変換する
        """
        return reduce(lambda x,y: x << 8 | y,byte_array) 
    
    def __to_byte_array(self,n,byte_cnt):
        """ int型をbyte配列に変換する
        """
        byte_array = array('B')
        for i in range(byte_cnt):
            byte_array.insert(0, (n >> (i * 8)) & 0xff)
        return byte_array
    
if __name__ == '__main__' :
    # 192.0.2.1 > 2.90.93.17
    # 2001:db8::1 > dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00
    key = array('B',range(32))
    obj = CryptoPAn(key)
    ip = '192.0.2.1'
    anonymizedIP = obj.anonymize(ip,32)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.17):",anonymizedIP 
    ip = '192.0.2.2'
    anonymizedIP = obj.anonymize(ip,24)
    print "Orig IP:",ip
    print "Ano IP(2.90.93.19):",anonymizedIP 
    ip = '2001:db8::1'
    anonymizedIP = obj.anonymize(ip,120)
    print "Orig IP:",ip
    print "Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00):",anonymizedIP 
    ip = '2001:db8::2'
    anonymizedIP = obj.anonymize(ip,120)
    print "Orig IP:",ip
    print "Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02):",anonymizedIP   

        

Orig IP: 192.0.2.1
Ano IP(2.90.93.17): 2.90.93.17
Orig IP: 192.0.2.2
Ano IP(2.90.93.19): 2.90.93.2
Orig IP: 2001:db8::1
Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e00): dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e01
Orig IP: 2001:db8::2
Ano IP(dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02): dd92:2c44:3fc0:ff1e:7ff9:c7f0:8180:7e02
